# Variance-level plots

### Classes and modules

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
import datetime
from IPython.display import display
import copy

#For plotting
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

plt.rcParams["image.origin"] = "lower"

import pycuda.driver as cuda

GPU Ocean-modules:

In [2]:
from gpuocean.utils import IPythonMagic, Common
from gpuocean.SWEsimulators import CDKLM16, ModelErrorKL

In [3]:
%cuda_context_handler gpu_ctx

In [4]:
gpu_stream = cuda.Stream()

Utils

In [5]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))
from utils.BasinInit import *
from utils.BasinAnalysis import *
from utils.BasinPlot import *

In [6]:
ls = [6, 7, 8, 9, 10]

In [7]:
grid_args_list = []

for l in ls:
    lvl_grid_args = initGridSpecs(l)
    grid_args_list.append( {
        "nx": lvl_grid_args["nx"],
        "ny": lvl_grid_args["ny"],
        "dx": lvl_grid_args["dx"],
        "dy": lvl_grid_args["dy"],
        } )

### Multi-Level Ensemble Sizes 

In [8]:
from utils.BasinAnalysis import *

In [9]:
source_path = "/home/florianb/havvarsel/multilevelDA/scripts/VarianceLevelsDA/Basin/2023-06-28T11_25_15sharedOmegaLocalised"

In [10]:
center_vars = np.load(source_path+"/center_vars_21600_L2norm.npy")
center_diff_vars = np.load(source_path+"/center_diff_vars_21600_L2norm.npy")

In [11]:
work_path = "/home/florianb/havvarsel/multilevelDA/scripts/PracticalCost/Basin/2023-06-29T14_54_02"

In [12]:
def raw2costsEnsemble(filename):
    rawCosts = np.load(filename)
    return np.mean(np.sort(rawCosts), axis=1)

In [13]:
costsPure = raw2costsEnsemble(work_path+"/costsPureEnsemble.npy")
costsPartnered = raw2costsEnsemble(work_path+"/costsPartneredEnsemble.npy")

In [14]:
analysis = Analysis(grid_args_list, center_vars, center_diff_vars, costsPure, costsPartnered)

In [15]:
tau = 1.5e-2

In [16]:
ML_Ne = analysis.optimal_Ne(tau=tau)
ML_Ne

array([6468, 1816,  511,  108,   19], dtype=int32)

In [17]:
analysis.work(ML_Ne)/costsPure[-1]

186.89689316758384

In [18]:
taus = [4.0e-2, 3.0e-2, 2.0e-2, 1.5e-2, 1.25e-2]

Nes = []

for tau in taus:
    ML_Ne = analysis.optimal_Ne(tau=tau)
    SL_Ne = np.ceil(analysis.work(ML_Ne)/costsPure[-1]).astype(int)

    Nes.append([ML_Ne, SL_Ne])

Nes

[[array([910, 256,  72,  16,   3], dtype=int32), 27],
 [array([1617,  454,  128,   27,    5], dtype=int32), 48],
 [array([3639, 1022,  288,   61,   11], dtype=int32), 106],
 [array([6468, 1816,  511,  108,   19], dtype=int32), 187],
 [array([9314, 2614,  736,  156,   27], dtype=int32), 269]]

DEBUG DEBUG DEBUG

In [19]:
from utils.BasinParameters import * 
from utils.BasinEnsembleInit import * 
from gpuocean.ensembles import MultiLevelOceanEnsemble

In [20]:
args_list = []

for l in ls:
    lvl_grid_args = initGridSpecs(l)
    args_list.append( {
        "nx": lvl_grid_args["nx"],
        "ny": lvl_grid_args["ny"],
        "dx": lvl_grid_args["dx"],
        "dy": lvl_grid_args["dy"],
        "gpu_ctx": gpu_ctx,
        "gpu_stream": gpu_stream,
        "boundary_conditions": Common.BoundaryConditions(2,2,2,2)
        } )

In [21]:
data_args_list = []
for l_idx in range(len(args_list)):
    data_args_list.append( make_init_steady_state(args_list[l_idx], a=steady_state_bump_a, bump_fractal_dist=steady_state_bump_fractal_dist) )


In [22]:
ML_ensemble = initMLensemble(Nes[0][0], args_list, data_args_list, sample_args, 
                             init_model_error_basis_args=init_model_error_basis_args, 
                             sim_model_error_basis_args=sim_model_error_basis_args, sim_model_error_time_step=sim_model_error_timestep)

MLOceanEnsemble = MultiLevelOceanEnsemble.MultiLevelOceanEnsemble(ML_ensemble)

MLOceanEnsemble.cleanUp()

In [23]:
ML_ensemble[0][0].t = 100

In [33]:
MLOceanEnsemble.ML_ensemble[0][0]